In [1]:
# Synchronizacja z dyskiem
from google.colab import drive
drive.mount("/content/drive")

%cd './drive/My Drive/Projekt/FashionMnist'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Projekt/FashionMnist


In [2]:
# Importy i sprawdzenie czy mamy CUDA
import torch
import torchvision

if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
dev

'cuda:0'

In [0]:
# parameters = {"batch_size": 100,
#               "kernel_size": 3,
#               "padding": 1, # dla kernel_size = 3 nie zmieni wielkości obrazka
#               "C": 4,
#               "n_blocks": 4,
#               "channels": [1, 16, 64, 128, 256],
#               "lr": 0.02,
#               "patience": 2,
#               "factor": 0.1,
#               "n_epochs": 40,
#               "dropouts": [0, 0.1, 0.2, 0.2],
#               "linear_size2": 64,
#               "name": "Fractal_v0.1"}
# Accuracy of the network on test images: 89.88 %

# parameters = {"batch_size": 100,
#               "kernel_size": 3,
#               "padding": 1, # dla kernel_size = 3 nie zmieni wielkości obrazka
#               "C": 4,
#               "n_blocks": 4,
#               "channels": [1, 16, 64, 128, 256],
#               "lr": 0.02,
#               "patience": 1,
#               "factor": 0.1,
#               "n_epochs": 30,
#               "dropouts": [0, 0.1, 0.1, 0.1],
#               "linear_size2": 64,
#               "name": "Fractal_v0.2"}
# Accuracy of the network on test images: 90.39 %

# parameters = {"batch_size": 100,
#               "kernel_size": 3,
#               "padding": 1, # dla kernel_size = 3 nie zmieni wielkości obrazka
#               "C": 4,
#               "n_blocks": 4,
#               "channels": [1, 16, 64, 128, 256],
#               "lr": 0.02,
#               "patience": 1,
#               "factor": 0.1,
#               "n_epochs": 30,
#               "dropouts": [0, 0.1, 0.1, 0.1],
#               "name": "Fractal_v0.3"}
# 29 loss: 0.212 train accuracy: 93.16 % 
# Accuracy of the network on test images: 90.77 %

# parameters = {"batch_size": 100,
#               "kernel_size": 3,
#               "padding": 1, # dla kernel_size = 3 nie zmieni wielkości obrazka
#               "C": 4,
#               "n_blocks": 4,
#               "channels": [1, 8, 64, 128, 256],
#               "lr": 0.02,
#               "patience": 1,
#               "factor": 0.1,
#               "n_epochs": 30,
#               "dropouts": [0, 0.1, 0.1, 0.1],
#               "name": "Fractal_v0.4"}
# Accuracy of the network on test images: 89.47 %

parameters = {"batch_size": 100,
              "kernel_size": 3,
              "padding": 1, # dla kernel_size = 3 nie zmieni wielkości obrazka
              "C": 4,
              "n_blocks": 4,
              "channels": [1, 16, 64, 128, 256],
              "lr": 0.02,
              "patience": 1,
              "factor": 0.1,
              "n_epochs": 35,
              "dropouts": [0, 0.1, 0.1, 0.2],
              "name": "Fractal_v0.31"}
# Accuracy of the network on test images: 91.25 %
# 32 loss: 0.225 train accuracy: 92.29 %

In [0]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# Katalog, w którym przechowujemy dane
root = "data"
batch_size = parameters["batch_size"]

# Transformacje wektorów wejściowych
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, ), (0.5, ))])

#ładowanie danych
train_dataset = dsets.FashionMNIST(root=root,
                            train=True,
                            transform=transform,
                            download=True)

test_dataset = dsets.FashionMNIST(root=root,
                            train=False,
                            transform=transform)

trainloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                batch_size=batch_size,
                                                shuffle=True)

testloader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                batch_size=batch_size,
                                                shuffle=True) 

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
kernel_size = parameters["kernel_size"]
padding = parameters["padding"]

In [0]:
import numpy as np

class DropPath(nn.Module):
  # Klasa implementująca drop-path opisany w pracy (dział 3.1)
  # Z tym samym prawdopodobiestwem wybierana jest strategia
  # "Local" i "Global"

  # Parametr `p` określa prawdopodobieństwo, z jakim każdy z inputów
  # będzie odrzucony, przy czym przynajmniej jedna ze ścieżek musi zostać
  # wybrana. Parametr jest wykorzystywany jedynie w strategii "Local"
  # `C` oznacza liczbę kolumn w jednym bloku sieci Fractal
  def __init__(self, p, C):
    super(DropPath, self).__init__()
    self.C = C
    self.p = p

  def prepare(self):
    self.is_global = np.random.choice([True, False])
    if self.is_global:
      self.allowed = np.random.choice(np.arange(1, self.C + 1))

  # TODO: wymyślić coś żeby w przypadku local była przynajmniej jedna
  # aktywna ścieżka
  def forward(self, x, column = None):
    if not self.training:
      return x
    if self.is_global:
      if column == self.allowed:
        return x
      else:
        return torch.zeros_like(x)
    if np.random.random() < self.p:
      return torch.zeros_like(x)
    return x

In [0]:
'''
Szczegóły implementacji są w dziale 3.3.
* "As now standard, we employ batch normalization together with each conv layer 
  (convolution, batch norm, then ReLU)."
'''

class Fractal(nn.Module):
  # Klasa implementuje blok "f_C" z pracy (dział 3)

  # parametr `n_channels` określa liczbę kanałów pomiędzy
  # kolejnymi `f_i`; np dla C = 1 mamy
  # n_channels = [in_channels, out_channels]
  # dropPath - instancja klasy DropPath w bloku, do którego należy Fractal
  def __init__(self, C, in_channels, out_channels, dropPath, dropout):
    super(Fractal, self).__init__()
    self.C = C
    self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
    self.bn2d = nn.BatchNorm2d(out_channels)
    self.dropPath = dropPath
    self.dropout = nn.Dropout2d(dropout)

    if C > 1:
      self.fC1 = Fractal(C - 1, in_channels, out_channels, dropPath, dropout)
      self.fC2 = Fractal(C - 1, out_channels, out_channels, dropPath, dropout)
    
  def forward(self, x):
    x1 = self.dropPath(F.relu(self.bn2d(self.conv2d(x))), self.C)
    x1 = self.dropout(x1)
    if self.C > 1:
      x2 = self.fC2(self.fC1(x))
      x1 = torch.mean(torch.stack([x1, x2]), 0) # join
    return x1

In [0]:
class Block(nn.Module):
  # Klasa implementuje budowę bloku, oraz zawiera w sobie rekurencyjne
  # tworzenie kolejnego

  # C jest parametrem określającym liczbę kolumn w danym bloku
  # n_block oznacza numer bloku (ten na samym dole na numer 0)
  # channels określa liczbę kanałów kolejno wejściowych i wyjściowych
  #   każdego bloku
  def __init__(self, C, n_block, channels, p, dropouts):
    super(Block, self).__init__()

    self.dropPath = DropPath(p[0], C)
    self.fractal = Fractal(C, channels[0], channels[1], self.dropPath, dropouts[0])
    self.n_block = n_block
    
    if self.n_block > 0:
      self.next_block = Block(C, n_block - 1, channels[1:], p[1:], dropouts[1:])

    self.pool = nn.MaxPool2d(2, 2)

  def forward(self, x):
    self.dropPath.prepare()
    x = self.pool(self.fractal(x))
    if self.n_block > 0:
      x = self.next_block(x)
    return x

In [0]:
class FractalNet(nn.Module):
  # C jest parametrem określającym liczbę kolumn w danym bloku
  # n_blocks oznacza liczbę bloków
  # channels określa liczbę kanałów kolejno wejściowych i wyjściowych
  #   każdego bloku
  # len(channels) = n_blocks + 1
  def __init__(self, C, n_blocks, channels, droppath_p, dropouts):
    super(FractalNet, self).__init__()

    if len(channels) != n_blocks + 1:
      raise RuntimeError("Niepoprawny rozmiar tablicy `channels`.")
    if len(droppath_p) != n_blocks:
      raise RuntimeError("Niepoprawny rozmiar tablicy `droppath_p`.")

    self.block = Block(C, n_blocks - 1, channels, droppath_p, dropouts)

    image_size = 28
    output_size = int(image_size / int(2 ** n_blocks))
    self.linearSize1 = (output_size ** 2) * channels[-1]
    # self.linearSize2 = parameters["linear_size2"]

    self.dense1 = nn.Linear(self.linearSize1, 10)
    # self.dense1 = nn.Linear(self.linearSize1, self.linearSize2)
    # self.dense2 = nn.Linear(self.linearSize2, 10)

  def forward(self, x):
    x = self.block(x)
    x = x.view(-1, self.linearSize1)
    x = self.dense1(x)
    # x = self.dense2(x)
    return x

In [0]:
net = FractalNet(parameters["C"], 
                 parameters["n_blocks"], 
                 parameters["channels"], 
                 [0.15, 0.15, 0.15, 0.15],
                 parameters["dropouts"])
# jeżeli można, używamy CUDA
if torch.cuda.is_available():
  net.to("cuda:0")

In [0]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=parameters["lr"], momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=parameters["factor"],
                              patience=parameters["patience"], verbose=True)


def accuracy(model, dataloader):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in dataloader:
          images, labels = data
          if torch.cuda.is_available():
            images = images.to("cuda:0")
            labels = labels.to("cuda:0")
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  return 100 * correct / total

In [0]:
save_path = "saved_models/" + parameters["name"]

In [0]:
n_epochs = parameters["n_epochs"] + 10
best_accuracy = 0

for epoch in range(n_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    running_loss_n = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if torch.cuda.is_available():
          inputs = inputs.to("cuda:0")
          labels = labels.to("cuda:0")


        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_loss_n += 1
    epoch_accuracy = accuracy(net, trainloader)
    print('%d loss: %.3f train accuracy: %.2f %%' %
                   (epoch + 1, running_loss / running_loss_n, 
                    epoch_accuracy))
    scheduler.step(epoch_accuracy)

    if best_accuracy < epoch_accuracy:
      best_accuracy = epoch_accuracy
      torch.save(net.state_dict(), save_path)

print('Finished Training')

1 loss: 1.098 train accuracy: 73.60 %
2 loss: 0.595 train accuracy: 80.11 %
3 loss: 0.520 train accuracy: 83.50 %
4 loss: 0.444 train accuracy: 85.58 %
5 loss: 0.409 train accuracy: 85.61 %
6 loss: 0.401 train accuracy: 86.89 %
7 loss: 0.356 train accuracy: 88.32 %
8 loss: 0.344 train accuracy: 87.76 %
9 loss: 0.347 train accuracy: 88.59 %
10 loss: 0.330 train accuracy: 89.16 %
11 loss: 0.300 train accuracy: 88.70 %
12 loss: 0.296 train accuracy: 89.42 %
13 loss: 0.305 train accuracy: 89.49 %
14 loss: 0.298 train accuracy: 90.19 %
15 loss: 0.289 train accuracy: 90.41 %
16 loss: 0.274 train accuracy: 90.47 %
17 loss: 0.284 train accuracy: 90.32 %
18 loss: 0.265 train accuracy: 90.52 %
19 loss: 0.287 train accuracy: 90.29 %
20 loss: 0.249 train accuracy: 90.60 %


In [0]:
net.load_state_dict(torch.load(save_path))
print('Accuracy of the network on test images: %.2f %%' % (
    accuracy(net, testloader)))